### FIFA 2019 - Prediction of player's sprint

Data from Kaggle: https://www.kaggle.com/karangadiya/fifa19

Model From: https://hackernoon.com/using-a-multivariable-linear-regression-model-to-predict-the-sprint-speed-of-players-in-fifa-19-530618986e1c


### Linear Regression

Linear Regression can be summed up as an attempt to model the relationship between one or multiple independent variables and a particular outcome or dependent variable. For this algorithm to be effective, there must be a linear relationship between the independent and dependent variables. Applied to data were a moderate to strong correlation exists between two or more variables it can be a very useful starting point in predicting the value of one outcome by finding the line that best fits/predicts an outcome.

## Y = MX + B

The math behind this is fairly simple, particularly where you are only looking at one independent variable. Y represents the outcome, or the dependent variable, while m denotes the slope, x the independent variable and b the y-intercept. Simply put, if you know the slope of the line and the value of the independent variable you can predict the outcome, assuming a linear relationship exists between x and y.

In my case however, I am going to be looking at multiple independent variables therefore the formula required changes slightly.

## F(x) = A +(B1*X1) +(B2*X2)+(B3*X3)+(B4*X4)...+(Bn*Xn)

With this formula I am assuming that there are (n) number of independent variables that I am considering. In this context F(x) is the predicted outcome of this linear model, A is the Y-intercept, X1-Xn are the predictors/independent variables, B1-Bn = the regression coefficients (comparable to the slope in the simple linear regression formula). Plugging the appropriate numbers in this formula would give me a prediction of an outcome, in this case the sprint speed of a player on FIFA19.

### Import libraries

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats  as stats

In [5]:
fifa_dataset = pd.read_csv("data.csv")
fifa_dataset.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


### Data Cleanup

I started off with a few assumptions, I assumed that sprint speed would be largely influenced by height, weight, age, acceleration stats and possibly the ratio between a player's weight and height. Upon observation of the data set I noticed that the heights and weights were recorded in string format (e.g 5'11 and 180lbs), additionally as someone who is more accustomed to the metric system I wanted to change these measurements to centimetres and kilograms respectively.

In [19]:
fifa_dataset.columns

# Height
#fifa_dataset['Height']= fifa_dataset.Height.str.replace("'",".").apply(lambda x: float(x)*30.48).dropna()
print(fifa_dataset.Height.isnull().sum())
#fifa_dataset.isnull().sum()
fifa_dataset["Loaned From"]

48


0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
5                NaN
6                NaN
7                NaN
8                NaN
9                NaN
10               NaN
11               NaN
12               NaN
13               NaN
14               NaN
15               NaN
16               NaN
17               NaN
18               NaN
19               NaN
20               NaN
21               NaN
22               NaN
23               NaN
24               NaN
25               NaN
26               NaN
27               NaN
28       Real Madrid
29               NaN
            ...     
18177            NaN
18178            NaN
18179            NaN
18180            NaN
18181            NaN
18182            NaN
18183            NaN
18184            NaN
18185            NaN
18186            NaN
18187            NaN
18188            NaN
18189            NaN
18190            NaN
18191            NaN
18192            NaN
18193        